In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select




from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import time
import traceback

In [2]:
import pandas as pd
from datetime import datetime


pos = 1
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())


,login,senha,ubs
0,joana,05216791418,borges
1,agda,09947741419,NaN


In [3]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [4]:
def people():
    
    openEpHealph()
        
    print(f'Extração iniciada as: {datetime.now()}')

    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")

    children = nav.find_element(By.ID, 'maxAge')
    nav.execute_script("arguments[0].setAttribute('value', '4.0')",children)
    time.sleep(1)
    
#     hypertension = nav.find_element(By.XPATH, '//*[@id="img_hipertensaoFlag"]').click()
#     time.sleep(1)

#     diabetes = nav.find_element(By.XPATH, '//*[@id="img_diabetesFlag"]').click()
#     time.sleep(1)

#     pregnant = nav.find_element(By.XPATH, '//*[@id="img_gestanteFlag"]').click()
#     time.sleep(1)

    """
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    race.send_keys(Keys.ENTER)
    """

    time.sleep(1)

    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()

    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [5]:
def get_table():
    
    list_links = []
    people()
    
    print(f'Iniciando lista de links: {datetime.now()}')

    exit = False
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    link = peoples.find_elements(By.TAG_NAME, 'a') 
    
    
    
    while exit is False:
        try:
            for a in link:
                
                list_links.append(a.get_attribute('href'))
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
            if ElementClickInterceptedException:    
                traceback.print_exc()
                
            if NoSuchElementException: 
                traceback.print_exc() 
            print('acabaram as paginas')
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            link = peoples.find_elements(By.TAG_NAME, 'a') 

    
    list_links = list(dict.fromkeys(list_links))
    print(f'lista de links finalizada as: {datetime.now()}')
    
    return list_links    


In [6]:
def get_individual_info():
    
    individual = get_table()
    
#   name sex b_date cpf cns has dm med_rec race m_name, proffisional, unity
    name, sex, b_date, cpf, cns, med_rec, race, m_name, profissional, unity = [], [], [], [], [], [], [], [], [], []
    
    
    print(f'Iniciando individuos : {datetime.now()}')
    for i in individual:
        
        time.sleep(1)
        
        nav.get(i)
        
        i_name = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/h5').text
        name.append(i_name)
        
        i_sex = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[1]').text
        i_sex = i_sex.split(',')
        sex.append(i_sex[0].upper())
        
        i_bdate = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[4]/td[2]').text
        i_bdate = i_bdate.split(";")
        b_date.append(i_bdate[0])
        
        i_cpf = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[2]').text
        cpf.append(i_cpf)
        
        i_cns = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        i_cns = i_cns.split(':')
        cns.append(i_cns[1])
        
        i_medrec = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
        med_rec.append(i_medrec)
        
        i_race = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[5]/td[2]').text
        race.append(i_race.upper())
        
        i_mname = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[6]/td[2]').text
        m_name.append(i_mname)
        
        i_prof = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[3]/div/div/div/div/div[1]/div/div[2]/div/div/table/tbody/tr[4]/td[2]').text
        profissional.append(i_prof)
        
        try:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[3]').text
        
        except NoSuchElementException:
            i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
                                            
        i_uni = i_uni.split(':')
        unity.append(i_uni[1])
        
        time.sleep(1)

    print(f'Finalizando individuos : {datetime.now()}')

    dict_individual = {"name": name, "sex":sex, "b_date":b_date, "cpf":cpf, "cns":cns, "med_rec":med_rec, "race":race, "m_name":m_name, "profissional":profissional, "unity":unity, "link":individual}
    df_individual = pd.DataFrame(data=dict_individual)    
    
    print(f'Extração finalizada as: {datetime.now()}')    
    return df_individual

# indi = get_indivivual_info()

In [7]:
children = get_individual_info()


/tmp/ipykernel_22755/3963594726.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())


Extração iniciada as: 2024-01-24 11:02:00.295284
Iniciando lista de links: 2024-01-24 11:02:18.984176


Traceback (most recent call last):
  File "/tmp/ipykernel_22755/2182560134.py", line 23, in get_table
    page.click()
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webelement.py", line 94, in click
    self._execute(Command.CLICK_ELEMENT)
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webelement.py", line 403, in _execute
    return self._parent.execute(command, params)
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webdriver.py", line 440, in execute
    self.error_handler.check_response(response)
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/errorhandler.py", line 245, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.ElementClickInterceptedException: Message: Element <a class="page-link" href="#"> is not clickable at point (1114,702) because another element <li id="individuo-list-d

acabaram as paginas
lista de links finalizada as: 2024-01-24 11:02:52.391194
Iniciando individuos : 2024-01-24 11:02:52.391211
Finalizando individuos : 2024-01-24 11:11:19.740912
Extração finalizada as: 2024-01-24 11:11:19.744294


In [10]:
display(children)
children.to_excel('teste.xlsx', index=False)
children = children[['cpf', 'name', 'b_date', 'sex', 'race', 'm_name', 'link']]

,cpf,name,b_date,sex,race,m_name,med_rec,link
0,16281687493,ABNER RYAN PEREIRA DA SILVA,05/02/2019,MASCULINO,BRANCA,GEORGIA PRISCILA PEREIRA DA SILVA INACIO,662,https://afogadosdaingazeira.ephealth.com.br/po...
1,18404115460,AGATHA HELENNA DA SILVA,05/01/2023,FEMININO,BRANCA,KARLA DANIELE ALVES DA SILVA,186,https://afogadosdaingazeira.ephealth.com.br/po...
2,16550578477,ANA ANGELA MARTINS DOS SANTOS,16/06/2019,MASCULINO,BRANCA,ANA ANGELA MARTINS DOS SANTOS,1516,https://afogadosdaingazeira.ephealth.com.br/po...
3,24/10/2023,ANA REBECA SILVA GOMES,07/06/2019,FEMININO,PARDA,MARIA EDUARDA DA SILVA,1226,https://afogadosdaingazeira.ephealth.com.br/po...
4,17188153442,ANDRE VINICIUS DE ARAUJO SILVA,14/05/2020,MASCULINO,BRANCA,JOSLLAYNE JOICE DEARAUJO SOARES SILVA,986,https://afogadosdaingazeira.ephealth.com.br/po...
...,...,...,...,...,...,...,...,...
103,26/07/2023,WALLCE GAEL FERREIRA SILVA,03/06/2023,MASCULINO,BRANCA,AMANDA GABRIELA FERREIRA DANTAS,1260,https://afogadosdaingazeira.ephealth.com.br/po...
104,16684169482,YAGO LUCAS CARVALHO SILVA,03/09/2019,MASCULINO,PARDA,MARIA KELY DA SILVA CARVALHO,1140,https://afogadosdaingazeira.ephealth.com.br/po...
105,16542988408,YASMIM RAMOS DE ALMEIDA,14/05/2019,FEMININO,BRANCA,MARIA JULIANA RAMOS,855,https://afogadosdaingazeira.ephealth.com.br/po...
106,16777211462,YASMIN VITÓRIA DA SILVA,17/10/2019,FEMININO,BRANCA,GENILDA SIMÃO DA SILVA,1068,https://afogadosdaingazeira.ephealth.com.br/po...


In [16]:
display(children)
children.to_excel(f'{data_e_hora}_{autentica['ubs'][1]}_children.xlsx', index=False)

,cpf,name,b_date,sex,race,m_name,link
0,16281687493,ABNER RYAN PEREIRA DA SILVA,05/02/2019,MASCULINO,BRANCA,GEORGIA PRISCILA PEREIRA DA SILVA INACIO,https://afogadosdaingazeira.ephealth.com.br/po...
1,18404115460,AGATHA HELENNA DA SILVA,05/01/2023,FEMININO,BRANCA,KARLA DANIELE ALVES DA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
2,16550578477,ANA ANGELA MARTINS DOS SANTOS,16/06/2019,MASCULINO,BRANCA,ANA ANGELA MARTINS DOS SANTOS,https://afogadosdaingazeira.ephealth.com.br/po...
3,24/10/2023,ANA REBECA SILVA GOMES,07/06/2019,FEMININO,PARDA,MARIA EDUARDA DA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
4,17188153442,ANDRE VINICIUS DE ARAUJO SILVA,14/05/2020,MASCULINO,BRANCA,JOSLLAYNE JOICE DEARAUJO SOARES SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
...,...,...,...,...,...,...,...
103,26/07/2023,WALLCE GAEL FERREIRA SILVA,03/06/2023,MASCULINO,BRANCA,AMANDA GABRIELA FERREIRA DANTAS,https://afogadosdaingazeira.ephealth.com.br/po...
104,16684169482,YAGO LUCAS CARVALHO SILVA,03/09/2019,MASCULINO,PARDA,MARIA KELY DA SILVA CARVALHO,https://afogadosdaingazeira.ephealth.com.br/po...
105,16542988408,YASMIM RAMOS DE ALMEIDA,14/05/2019,FEMININO,BRANCA,MARIA JULIANA RAMOS,https://afogadosdaingazeira.ephealth.com.br/po...
106,16777211462,YASMIN VITÓRIA DA SILVA,17/10/2019,FEMININO,BRANCA,GENILDA SIMÃO DA SILVA,https://afogadosdaingazeira.ephealth.com.br/po...
